In [ ]:
import websocket
import json
from urllib.parse import quote

In [ ]:
ws = websocket.WebSocket()
website_id='google.com'
ws.connect(f'wss://plankton-app-5rfza.ondigitalocean.app/websites?scraping={website_id}')
ws.recv()

In [ ]:
ws.close()

In [ ]:
ws = websocket.WebSocket()
page_id = 'https://betway.com/es/sports/evt/10682299","websiteId'
website_id='betway.com'
# uri = 'wss://plankton-app-5rfza.ondigitalocean.app'
uri='ws://localhost:8000'
ws.connect(f'{uri}/games/{website_id}?game_id={page_id}')
ws.recv()

In [194]:
from core.models import r, Game
from redis.commands.search.query import Query, NumericFilter
from  redis.commands.search import aggregation
from redis.commands.search import reducers
from datetime import datetime, timezone

In [ ]:
r.ft('idxGames').search(Query('@scraping:{true}').sort_by('lastScraping'))

In [ ]:
from bs4 import BeautifulSoup

In [199]:
from core.webdriver import init_driver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from core.utils import scroll_down
from core.models import Bet
from time import sleep
driver = init_driver()
url= 'https://www.dafabet.com/es/dfgoal/sports/240-football'
driver.get(url)


In [198]:
for elm in driver.find_elements(By.CSS_SELECTOR, '.expand .subitems h4.parent a'):
        print(elm.get_attribute('href'))
    

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=108.0.5359.99)
Stacktrace:
Backtrace:
	(No symbol) [0x008CF243]
	(No symbol) [0x00857FD1]
	(No symbol) [0x0074D04D]
	(No symbol) [0x00732D7A]
	(No symbol) [0x0079BE7B]
	(No symbol) [0x007AC196]
	(No symbol) [0x00798386]
	(No symbol) [0x0077163C]
	(No symbol) [0x0077269D]
	GetHandleVerifier [0x00B69A22+2655074]
	GetHandleVerifier [0x00B5CA24+2601828]
	GetHandleVerifier [0x00978C0A+619850]
	GetHandleVerifier [0x00977830+614768]
	(No symbol) [0x008605FC]
	(No symbol) [0x00865968]
	(No symbol) [0x00865A55]
	(No symbol) [0x0087051B]
	BaseThreadInitThunk [0x75967BA9+25]
	RtlInitializeExceptionChain [0x775DBB9B+107]
	RtlClearBits [0x775DBB1F+191]
